<a href="https://colab.research.google.com/github/JHyunjun/DQTGAN/blob/main/WGANGP_VGGLOSS_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, utils
from torchvision.models import vgg16
from torch.utils.data import DataLoader
from torch.autograd import Variable, grad

# 설정값
batch_size = 64
epochs = 10
latent_dim = 100
lambda_gp = 10
n_critic = 5

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 로드
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

downsample = transforms.Resize(16)

dataset = datasets.CIFAR10(root='./', download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input is 3 x 16 x 16
            nn.ConvTranspose2d(3, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            # Size now is 256 x 32 x 32
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # Size now is 128 x 64 x 64
            nn.ConvTranspose2d(128, 3, 3, 1, 1, bias=False),
            nn.Tanh()
            # Output size is 3 x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Input size is 3 x 64 x 64
            nn.Conv2d(3, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Size now is 128 x 32 x 32
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # Size now is 256 x 16 x 16
            nn.Conv2d(256, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            # Output is a scalar probability
        )

    def forward(self, input):
        return self.main(input).view(-1)

# Pretrained VGG for perceptual loss
class VGGPerceptualLoss(nn.Module):
    def __init__(self):
        super(VGGPerceptualLoss, self).__init__()
        model = vgg16(pretrained=True)
        features = model.features
        self.to_relu_1_2 = nn.Sequential()
        for x in range(4):
            self.to_relu_1_2.add_module(str(x), features[x])
        self.to_relu_1_2 = self.to_relu_1_2.eval()

    def forward(self, input, target):
        input = (input + 1) / 2
        target = (target + 1) / 2
        return torch.nn.functional.l1_loss(self.to_relu_1_2(input), self.to_relu_1_2(target))

# WGAN-GP gradient penalty
def gradient_penalty(critic, real, fake, device):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = alpha * real + ((1 - alpha) * fake)
    mixed_scores = critic(interpolated_images)

    gradient = grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    penalty = torch.mean((gradient_norm - 1)**2)
    return penalty

# 모델 생성
G = Generator().to(device)
D = Discriminator().to(device)
vgg_loss = VGGPerceptualLoss().to(device)

# Optimizers
G_optimizer = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
D_optimizer = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training Loop
for epoch in range(epochs):
    for i, (real, _) in enumerate(dataloader):
        real = real.to(device)
        real_downsampled = downsample(real).to(device)

        # Train Discriminator
        D_optimizer.zero_grad()

        fake = G(real_downsampled)
        real_score = D(real)
        fake_score = D(fake)

        gp = gradient_penalty(D, real, fake, device)
        d_loss = -(torch.mean(real_score) - torch.mean(fake_score)) + lambda_gp * gp

        d_loss.backward(retain_graph = True)
        D_optimizer.step()

        # Train Generator
        if i % n_critic == 0:
            G_optimizer.zero_grad()

            fake_score = D(fake)
            perceptual_loss = vgg_loss(fake, real)
            g_loss = -torch.mean(fake_score) + perceptual_loss

            g_loss.backward()
            G_optimizer.step()

    print(f'Epoch [{epoch}/{epochs}] d_loss: {d_loss.item()} g_loss: {g_loss.item()}')

    if epoch % 5 == 0:
        utils.save_image(fake.data[:25], f"{epoch}.png", nrow=5, normalize=True)


In [ ]:
import matplotlib.pyplot as plt

# 테스트 데이터 로드
test_dataset = datasets.CIFAR10(root='./', train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# 테스트 데이터셋에서 이미지 선택
test_images = next(iter(test_dataloader))[0][:25].to(device)  # First batch
test_images = downsample(test_images)  # Downsample the test images to 16x16

def show_generated_img():
    G.eval()  # Set the generator to evaluation mode
    with torch.no_grad():
        test_images_fake = G(test_images)
    G.train()  # Set the generator to training mode

    grid = utils.make_grid(test_images_fake, nrow=5, normalize=True).permute(1, 2, 0)
    plt.figure(figsize=(10,10))
    plt.imshow(grid.detach().cpu().numpy())
    plt.axis('off')
    plt.show()

# Show generated images
show_generated_img()
